# Introduction

In this example we'll use the Fashion MNIST dataset to do basic classification. 

# Dependencies

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Load datasets

In [8]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



Use data loaders to iterate over the data

In [9]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Create the model

In [13]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

print(f"Using device {device}")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using device cuda
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Train the model

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        #Back propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Run some training rounds

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------
loss: 2.306046 [   64/60000]
loss: 2.299242 [ 6464/60000]
loss: 2.279033 [12864/60000]
loss: 2.266370 [19264/60000]
loss: 2.256443 [25664/60000]
loss: 2.225756 [32064/60000]
loss: 2.230556 [38464/60000]
loss: 2.197742 [44864/60000]
loss: 2.193237 [51264/60000]
loss: 2.168344 [57664/60000]
Test Error: 
Accuracy: 43.7%, Avg loss: 2.159248 

Epoch 2
---------------------
loss: 2.168950 [   64/60000]
loss: 2.163776 [ 6464/60000]
loss: 2.111214 [12864/60000]
loss: 2.119399 [19264/60000]
loss: 2.073436 [25664/60000]
loss: 2.010857 [32064/60000]
loss: 2.043927 [38464/60000]
loss: 1.958781 [44864/60000]
loss: 1.965903 [51264/60000]
loss: 1.902760 [57664/60000]
Test Error: 
Accuracy: 54.9%, Avg loss: 1.897413 

Epoch 3
---------------------
loss: 1.928336 [   64/60000]
loss: 1.904017 [ 6464/60000]
loss: 1.792589 [12864/60000]
loss: 1.824909 [19264/60000]
loss: 1.717771 [25664/60000]
loss: 1.662383 [32064/60000]
loss: 1.696005 [38464/60000]
loss: 1.584957 [44864/600

# Save the model

In [19]:
torch.save(model.state_dict(), "fashion_mnist_model.pth")
print("Saved PyTorchModel State to fashion_mnist_model.pth")

Saved PyTorchModel State to fashion_mnist_model.pth


# Load the model

In [21]:
stored_model = NeuralNetwork().to(device)
stored_model.load_state_dict(torch.load("fashion_mnist_model.pth"))

<All keys matched successfully>

Try the stored model

In [24]:
classes = [
    "t-shirt",
    "pants",
    "pullover",
    "dress",
    "coat",
    "sandal",
    "shirt",
    "sneaker",
    "bag",
    "ankle boot",
]

stored_model.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    print(f"{pred}")
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([[-1.9974, -2.7303, -0.8475, -2.0943, -1.1892,  2.3736, -1.0032,  2.6973,
          1.6911,  3.0828]], device='cuda:0')
Predicted: "ankle boot", Actual: "ankle boot"
